In [1]:
import pandas as pd
import numpy as np

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score

import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
import polars as pl
from pathlib import Path
from glob import glob
import json
import joblib
import warnings
warnings.filterwarnings("ignore")




In [2]:
with open('../IND-WhoIsWho/pid_to_info_all.json', 'r') as file:
    pid = json.load(file)

In [3]:

train = pd.read_feather('data/train.feather')
valid = pd.read_feather('data/valid.feather')
test = pd.read_feather('data/test.feather')

train['dataset_mode'] = 0
valid['dataset_mode'] = 1
test['dataset_mode'] = 2

data = pd.concat([train,valid,test]).reset_index(drop = True)



In [ ]:
df_oof = pd.read_feather('oof2/cat_oof.feather')
del df_oof['label_prob']

temp =  pd.read_feather('oof2/lgb_oof.feather')
df_oof['lgb_prob'] = temp['label_prob']

temp =  pd.read_feather('oof2/xgb_oof.feather')
df_oof['xgb_prob'] = temp['label_prob']


temp =  pd.read_feather('oof2/cat_oof.feather')
df_oof['cat_prob'] = temp['label_prob']


temp =  pd.read_feather('oof2/feat_select_lgb_oof.feather')
df_oof['lgb_prob_2'] = temp['label_prob']

temp =  pd.read_feather('oof2/feat_select_xgb_oof.feather')
df_oof['xgb_prob_2'] = temp['label_prob']

temp =  pd.read_feather('oof2/feat_select_cat_oof.feather')
df_oof['cat_prob_2'] = temp['label_prob']


temp =  pd.read_feather('oof2/gnn_oof.feather')
df_oof['gnn_prob'] = temp['label_prob']


prediction = pd.read_feather('oof2/cat_prediction.feather')
del prediction['label_prob']

temp =  pd.read_feather('oof2/lgb_prediction.feather')
prediction['lgb_prob'] = temp['label_prob']

temp =  pd.read_feather('oof2/xgb_prediction.feather')
prediction['xgb_prob'] = temp['label_prob']


temp =  pd.read_feather('oof2/cat_prediction.feather')
prediction['cat_prob'] = temp['label_prob']


temp =  pd.read_feather('oof2/feat_select_lgb_prediction.feather')
prediction['lgb_prob_2'] = temp['label_prob']

temp =  pd.read_feather('oof2/feat_select_xgb_prediction.feather')
prediction['xgb_prob_2'] = temp['label_prob']

temp =  pd.read_feather('oof2/feat_select_cat_prediction.feather')
prediction['cat_prob_2'] = temp['label_prob']


temp =  pd.read_feather('oof2/gnn_prediction.feather')
prediction['gnn_prob'] = temp['label_prob']



temp = pd.concat([df_oof,prediction]).reset_index(drop = True)
del temp['label']

In [ ]:
edges = joblib.load('edges_all.pkl')
data['id'] = data.index

In [ ]:
edges_0 = dict(edges.groupby(0)[1].agg(list))
edges_1 = dict(edges.groupby(1)[0].agg(list))

In [ ]:
weight = [3,3,1,1,1,1,8]
#weight = [0,0,0,0,0,0,1]
weight = np.array(weight) / sum(weight)
df_oof['label_prob'] = 0
prediction['label_prob'] = 0

ff = [ 'lgb_prob', 'xgb_prob', 'cat_prob','lgb_prob_2', 'xgb_prob_2', 'cat_prob_2','gnn_prob']
for i in range(len(ff)):
    df_oof['label_prob'] += df_oof[ff[i]] * weight[i]
    prediction['label_prob'] += prediction[ff[i]] * weight[i]

In [ ]:
df_oof['id'] = data['id'].values[:df_oof.shape[0]]
prediction['id'] = data['id'].values[df_oof.shape[0]:]
t_dict = dict(df_oof[['id','label_prob']].values)
t_dict.update(dict(prediction[['id','label_prob']].values))

In [ ]:

df_oof['neighbour_0'] = df_oof['id'].map(edges_0)
df_oof.loc[df_oof['neighbour_0'].isna(),'neighbour_0'] = df_oof.loc[df_oof['neighbour_0'].isna(),'id'].apply(lambda x:[x])
df_oof['neighbour_1'] = df_oof['id'].map(edges_1)
df_oof.loc[df_oof['neighbour_1'].isna(),'neighbour_1'] = df_oof.loc[df_oof['neighbour_1'].isna(),'id'].apply(lambda x:[x])
df_oof['neighbour_1'] = df_oof['neighbour_0'] + df_oof['neighbour_1']
df_oof['neighbour_1'] = df_oof['neighbour_1'].apply(lambda x:list(set(x)))

df_oof['neighbour_1'] = df_oof['neighbour_1'].apply(lambda x:[t_dict[f] for f in x])
df_oof['neighbour_1'] = df_oof['neighbour_1'].apply(lambda x:np.mean(x))


In [ ]:
prediction['neighbour_0'] = prediction['id'].map(edges_0)
prediction.loc[prediction['neighbour_0'].isna(),'neighbour_0'] = prediction.loc[prediction['neighbour_0'].isna(),'id'].apply(lambda x:[x])
prediction['neighbour_1'] = prediction['id'].map(edges_1)
prediction.loc[prediction['neighbour_1'].isna(),'neighbour_1'] = prediction.loc[prediction['neighbour_1'].isna(),'id'].apply(lambda x:[x])
prediction['neighbour_1'] = prediction['neighbour_0'] + prediction['neighbour_1']
prediction['neighbour_1'] = prediction['neighbour_1'].apply(lambda x:list(set(x)))

prediction['neighbour_1'] = prediction['neighbour_1'].apply(lambda x:[t_dict[f] for f in x])
prediction['neighbour_1'] = prediction['neighbour_1'].apply(lambda x:np.mean(x))

In [ ]:
prediction['label_prob_post'] = prediction['neighbour_1']*0.4 + prediction['label_prob']*0.6

In [ ]:

with open("IND-test-public/ind_test_author_submit.json") as f:
    submission=json.load(f)

In [ ]:
test_preds = prediction.loc[df_test['dataset_mode'] == 2,'label_prob_post'].to_list()
cnt=0
for id,names in submission.items():
    for name in names:
        submission[id][name]=test_preds[cnt]
        cnt+=1
with open('sub/B_baseline_608_3.json', 'w', encoding='utf-8') as f:
    json.dump(submission, f, ensure_ascii=False, indent=4)